https://neptune.ai/blog/select-model-for-time-series-prediction-task

https://towardsdatascience.com/anomaly-detection-in-time-series-sensor-data-86fd52e62538

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics import tsaplots

In [ ]:
data = pd.read_csv('PWHEAMTUSDM.csv', parse_dates=True, index_col=0)

data.plot()

print(adfuller(data))

data = data.diff().dropna()
data.plot()
print(adfuller(data))
fig = tsaplots.plot_acf(data, color='r', lags=20, zero=False)
plt.show()

Now the data is stationary and has no autocorelation.

We'll use 3 algorithms to detect anomalies:

1. Benchamark model: Interquartile Range (IQR)
2. K-Means clustering
3. Isolation forest

1. IQR

In [ ]:
q1_data, q3_data = data['PWHEAMTUSDM'].quantile([0.25, 0.75])
iqr_data = q3_data - q1_data

lower_bound = q1_data - 1.5 * iqr_data
upper_bound = q3_data + 1.5 * iqr_data

print(lower_bound, upper_bound)

Filtering out outliers

In [ ]:
data['anomaly1'] = ((data>upper_bound) | (data<lower_bound)).astype(int)


Plotting

In [ ]:
a = data[data['anomaly1'] == 1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(data['PWHEAMTUSDM'], color='blue', label='Normal')
_ = plt.plot(a['PWHEAMTUSDM'], linestyle='none', marker='X', color='red', markersize=12, label='Anomaly')
_ = plt.xlabel('Year')
_ = plt.ylabel('Wheat price in USD')
_ = plt.title('IQR based Anomalies')
_ = plt.legend(loc='best')
plt.show()

2.1. K-means Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(data.values)
labels = kmeans.predict(data.values)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

In [ ]:
#function which calculates distance between each point and the centroid of the closest cluster

def getDistanceBetweenPointsAndCentroid(data, model):
  distance = []
  data_indeces_as_ints = data.reset_index(drop=True)
  for i in data_indeces_as_ints.index:
    Xa = np.array(data_indeces_as_ints.loc[i])
    Xb = model.cluster_centers_[model.labels_[i]-1]
    distance.append(np.linalg.norm(Xa-Xb))
  return pd.Series(distance, index=data.index)

In [ ]:
# this value has to be tuned 
outliers_fractions = 0.13

distance = getDistanceBetweenPointsAndCentroid(data, kmeans)

number_of_outliers = int(outliers_fractions*len(distance))

threshold_distance = distance.nlargest(number_of_outliers).min()

data['anomaly2'] = (distance >= threshold_distance).astype(int)

Plotting

In [ ]:
a = data[data['anomaly2'] == 1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(data['PWHEAMTUSDM'], color='blue', label='Normal')
_ = plt.plot(a['PWHEAMTUSDM'], linestyle='none', marker='X', color='red', markersize=12, label='Anomaly')
_ = plt.xlabel('Year')
_ = plt.ylabel('Wheat price in USD')
_ = plt.title('K-means based Anomalies')
_ = plt.legend(loc='best')
plt.show()

2.2. K-mean 

3.1. Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
#this value has to be tuned
outliers_fractions = 0.2

model = IsolationForest(contamination=outliers_fractions)
model.fit(data.values)
data['anomaly3'] = model.predict(data.values)

Plotting

In [ ]:
data['anomaly3'] = pd.Series(data['anomaly3'].values, index=data.index)
a = data.loc[data['anomaly3'] == -1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(data['PWHEAMTUSDM'], color='blue', label='Normal')
_ = plt.plot(a['PWHEAMTUSDM'], linestyle='none', marker='X', color='red', markersize=12, label='Anomaly')
_ = plt.xlabel('Year')
_ = plt.ylabel('Wheat price in USD')
_ = plt.title('Isolation Forest based Anomalies')
_ = plt.legend(loc='best')
plt.show()


3.2. Isolation Forest with optimization

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

In [ ]:
true_labels = np.zeros(len(data))
true_labels[np.random.choice(len(data), size=int(0.2 * len(data)), replace=False)] = 1

param_grid = {
    'n_estimators': [100, 200],
    'max_samples': ['auto', 0.8, 0.9],
    'contamination': [0.1, 0.15, 0.2],
    'max_features': [1.0, 0.8, 0.9]
}

if_model = IsolationForest()
grid_search = GridSearchCV(if_model, param_grid, cv=5, scoring=make_scorer(f1_score), n_jobs=-1)
grid_search.fit(data.values, true_labels)

print("Best parameters found: ", grid_search.best_params_)
best_if_model = grid_search.best_estimator_

data['anomaly3optimized'] = best_if_model.predict(data.values)

# Plotting
data['anomaly3optimized'] = pd.Series(data['anomaly3optimized'].values, index=data.index)
a = data.loc[data['anomaly3optimized'] == -1] # anomaly
plt.figure(figsize=(18,6))
plt.plot(data['PWHEAMTUSDM'], color='blue', label='Normal')
plt.plot(a['PWHEAMTUSDM'], linestyle='none', marker='X', color='red', markersize=12, label='Anomaly')
plt.xlabel('Year')
plt.ylabel('Wheat price in USD')
plt.title('Isolation Forest (optimized) based Anomalies')
plt.legend(loc='best')
plt.show()